In [ ]:
!cp -r ../input/updated-code-3-1/* /kaggle/working/

In [ ]:
!pip install Cython h5py nltk nose numpy pandas scikit-learn scikit-optimize scipy tqdm lightfm RISE torch jupyter-contrib-nbextensions tensorflow tables protobuf

In [ ]:
#per ridere
!python run_compile_all_cython.py

In [ ]:
!pip install optuna

In [ ]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
import matplotlib.pyplot as plt
from scipy.stats import loguniform
import time
import optuna

In [ ]:
#from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.Similarity.Cython import Compute_Similarity_Cython
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.BaseRecommender import BaseRecommender
#from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
#----remove

In [ ]:
seed = 69
np.random.seed(seed)

In [ ]:
# Import training data
URM_path = "../input/data-books/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                header=0,
                                dtype={0:int, 1:int, 2:int},
                                engine='python')

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]

In [ ]:
# Import target users
target_path = "../input/data-books/data_target_users_test.csv"
target_dataframe= pd.read_csv(filepath_or_buffer=target_path,
                                header=0,
                                dtype={0:int},
                                engine='python')
target_dataframe.columns = ["user_id"]
target_dataframe

In [ ]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings

In [ ]:
# Call preprocess data function
ratings = preprocess_data(URM_all_dataframe)

In [ ]:
URM = sps.coo_matrix((ratings.interaction.values, (ratings.mapped_user_id.values, ratings.mapped_item_id.values)))

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout

In [ ]:
urm_train, urm_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from multiprocessing import cpu_count, Lock

In [ ]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        # Create a DataFrame from the current trial's results
        trial_df = pd.DataFrame([hyperparam_dict])

        # Use concat instead of append
        self.results_df = pd.concat([self.results_df, trial_df], ignore_index=True)

In [ ]:
cpu_count()

In [ ]:
def objective_function_SLIM(optuna_trial):

    recommender_instance = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train)
    recommender_instance.fit(
                             l1_ratio = optuna_trial.suggest_float("l1_ratio", 1e-3, 0.1),
                             alpha = optuna_trial.suggest_float("alpha", 1e-3, 1),
                             topK = optuna_trial.suggest_int("topK", 300, 1000),
                             workers = int(cpu_count())
                             #positive_only = optuna_trial.suggest_categorical("positive_only", [True, False]),
                            )

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [ ]:
optuna_study_SLIM = optuna.create_study(direction="maximize")

save_results_SLIM = SaveResults()

optuna_study_SLIM.optimize(objective_function_SLIM,
                      callbacks=[save_results_SLIM],
                      n_trials = 100)

In [ ]:
best = {'l1_ratio': 0.020774395637352183, 'alpha': 0.002593925519077253, 'topK': 619}

In [ ]:
#Trial 10 finished with value: 0.02977694828264403 and parameters: {'l1_ratio': 0.01732982416943757, 'alpha': 0.0010658461296225302, 'topK': 620}. Best is trial 10 with value: 0.02977694828264403.